## This assignment may be worked individually or in pairs. 
## Enter your name/names here:
    

In [1]:
# David Mao, dm46452
# Stephen Aigbomian

# Assignment 2: Naive Bayes and KNN classifier

In this assignment you'll implement the Naive Bayes and KNN classifier to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the same Diabetic Retinopathy data set which was used in the previous assignment on decision trees. The implementation details are up to you but, generally it is a good idea to divide your code up into helper functions.

In [2]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log
from random import shuffle

In [3]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Read the data from a CSV file. You may either put it into a list of `DataPoints` as you did on the previous assignment (class provided above), or you may choose to store it any any format you wish, like a Pandas dataframe, or any other format you'd like.

In [4]:
data_frame = pd.read_csv("messidor_features.txt", header=None)
data_frame.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1


## Part 1: Naive Bayes Classifier

Naive Bayes (NB) classifier is a simple probabilistic classifier that is based on applying the Bayes' theorem and assumes a strong (naive) independence between features. The Diabetic Retinopath data set contains both categorical and continuous features. Dealing with categorical features has been already been discussed in detail in class. Continuous attributes, on the other hand, are more interesting to handle. Most commonly, this is done by assuming normal probability distribution over the feature values or by binning the attribute values in a fixed number of bins. In this assignment you'll be implementing the binning approach. For each continuous attribute, you'll construct 3 equal sized bins. For example, feature 5 ranges from `[1 - 120]` the 3 bins that you'll construct will be `[1 - 40]`, `[41 - 80]`, `[81 - 120]`.

Q1. Implement a Naive Bayes classifier. Measure the accuracy of your classifier using 5-fold cross validation and display the confusion matrix. Also print the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [5]:
# your code goes here

binary_features = (0, 1, 18)
from collections import OrderedDict
class Naive_Bayes_Classifier:
    def __init__(self, data):
        self.data = data
        self.cond_pos = []
        self.cond_neg = []
    
    def compute_conditionals(self, classification, feature_index):
        data = self.data[self.data[19] == classification]
        if feature_index in binary_features:
            num_pos, num_neg = self.compute_binary_prop(data, feature_index)
            if classification:
                self.cond_pos.append({1: num_pos, 0: num_neg})
            else:
                self.cond_neg.append({1: num_pos, 0: num_neg})
        else:
            props = self.compute_cont_prop(data, feature_index)
            if classification:
                self.cond_pos.append(props)
            else:
                self.cond_pos.append(props)
    
    def compute_binary_prop(self, data, feature_index):
        length = data[feature_index].count()
        num_pos = data[data[feature_index] == 1].count()
        return (num_pos, length - num_pos)
    
    def compute_cont_prop(self, data, feature_index):
        feature_min = data[feature_index].min()
        feature_max = data[feature_index].max()
        bin_size = (feature_max - feature_min) / 3
        bins = []
        start = feature_min
        end = feature_min + bin_size
        for i in range(3):
            bins.append((start, end))
            start += bin_size
            end += bin_size
        props = OrderedDict()
        
        for i in range(3):
            cur_bin = bins[i]
            bin_start, bin_end = cur_bin
            if i < 2:
                props[(bin_start, bin_end)] = data[bin_start <= data[feature_index] & data[feature_index] < bin_end].count()
            else:
                props[(bin_start, bin_end)] = data[bin_start <= data[feature_index] & data[feature_index] <= bin_end].count()
        return props
    
    def train(self):
        for i in range(19):
            self.compute_conditionals(1, i);
            self.compute_conditionals(0, i)
    
    def calculate_prob(self, data_point, label):
        props = []
        if label:
            props = self.cond_pos
        else:
            props = self.cond_neg
        
        prob = 1.0
        for index, feature in enumerate(data_point):
            feature_props = self.cond_pos[index]
            if index in binary_features:
                if feature:
                    prob *= feature_props[1] / sum(feature_props.values())
                else:
                    prob *= feature_props[0] / sum(feature_props.values())
            else:
                for index, cur_bin in enumerate(feature_props.keys()):
                    bin_start, bin_end = cur_bin
                    if index < 2:
                        if bin_start <= feature < bin_end:
                            prob *= feature_props[cur_bin] / sum(feature_props.values())
                    else:
                        if bin_start <= feature <= bin_end:
                            prob *= feature_props[cur_bin] / sum(feature_props.values())
        
    def classify(self, data_point):
        prob_pos = self.calculate_prob(data_point, 1)
        prob_neg = self.calculate_prob(data_point, 0)
        return 1 if prob_pos > prob_neg else 0

def calcAccuracy(bayes, data):
    total = 0
    num_correct = 0
    for data_point in data:
        if bayes.classify(data_point) == data_point.label:
            num_correct += 1
        total += 1
    
    return num_correct / total

d = pd.read_csv("messidor_features.txt", header=None)
test_set_len = len(d) // 5
accuracies = []
for i in range(5):
# partition data into train_set and test_set
    train_set = d.loc[0:i * test_set_len] + d.loc[(i + 1) * test_set_len:len(d)]
    test_set = d.loc[i * test_set_len:(i + 1) * test_set_len]

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the classifier
    bayes = Naive_Bayes_Classifier(train_set)
    bayes.train()

    # calculate the accuracy of the classifier
    accuracy = calcAccuracy(bayes, test_set)
    accuracies.append(accuracy)
    print ('The accuracy on the test set ', i, ' is ', str(accuracy * 100.0))
avg_accuracy = sum(accuracies) / len(accuracies)
print ('The average accuracy on the data set is: ', str(avg_accuracy))

Training set size: 922
Test set size    : 231


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


## Part 2: K Nearest Neighbor (KNN) Classifier

The KNN classifier consists of two stages:-
- In the training stage, the classifier takes the training data and simply memorizes it
- In the test stage, the classifier compares the test data with the training data and simply returns the maximum occuring label of the k nearest data points.

The distance calculation method is central to the algorithm, typically Euclidean distance is used but other distance metrics like Manhattan distance can also be used. In this assignment you'll be implementing the classifier using the Euclidean distance metric. It is important to note that, Euclidean distance is very sensitive to the scaling of different attributes hence, before you can build your classifier you have to normalize the values of each feature in the data set.

Q2. Normalize the dataset so that each feature value lies between `[0-1]`.

In [ ]:
# your code goes here


Q3. Build your KNN classifier. 

In [ ]:
# your code goes here


Q4. Find the best value of k using 5-fold cross validation. In each fold of CV, divide your data into a training set and a validation set. Try k ranging from 1 to 10 and plot the accuracies using 5-fold CV. Use this plot to identify the best value of k (provide reasoning).

In [ ]:
# your code goes here


Q5. Now measure the accuracy of your classifier using 5-fold cross validation. In each fold of this CV, divide your data into a training set and a test set. The training set should get sent through your code for Q4, resulting in a value of k to use. Using that k, calculate an accuracy on the test set. You will average the accuracy over all 5 folds to obtain the final accuracy measurement. Print the accuracy as well as the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [ ]:
# your code goes here


### To explore further:

1) Use scikit-learn's NearestNeighbor classifier to classify the data. Compare those results to your own. The documentation can be found [here](http://scikit-learn.org/stable/modules/neighbors.html), specifically section 1.6.2 and the link to the [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier).

2) Use scikit-learn's Naive Bayes classifier to classify the data. Compare those results to your own. The documentation is found [here](http://scikit-learn.org/stable/modules/naive_bayes.html), specifically section 1.9.2 and the link to [MultinomialNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB). You will want to bin the continuous attributes, as you did above, and consider each bin to be a categorical value for that attribute. For example, feature 5 ranges from [1 - 120], and the 3 bins that you'll construct will be [1 - 40], [41 - 80], [81 - 120]. You can think of this as saying that feature 5 can take 1 of 3 values: low, med, or high. With binning, you have transformed all of your features into categorical features and the MultinomialNB version of Naive Bayes is what should be used for all categorical data. 